In [1]:
import numpy as np
import  pandas as pd
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [2]:
data = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/Assignments/Red_Wine_Quality/winequality-red.csv')

In [3]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [4]:
X = data
Y = data['quality']

In [5]:
print(X.shape, Y.shape)

(1599, 12) (1599,)


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 20)

In [7]:
def entropy(col):
    elements, counts = np.unique(col, return_counts=True)
    ent = 0.0
    for i in range(len(elements)):
        prob = counts[i] / len(col)
        ent -= prob * np.log2(prob)
    return ent

In [8]:
def divide_data(X_data, feat, value):
    X_left = X_data[X_data[feat] < value].reset_index(drop=True)
    X_right = X_data[X_data[feat] >= value].reset_index(drop=True)
    return X_left, X_right

In [9]:
def info_gain(X_data, feat, value):
    left, right = divide_data(X_data, feat, value)
    
    if len(left) == 0 or len(right) == 0:
        return 0
    
    p_left = len(left) / len(X_data)
    p_right = len(right) / len(X_data)
    
    return entropy(X_data['quality']) - (p_left * entropy(left['quality']) + p_right * entropy(right['quality']))

In [10]:
class Decision_Tree:
    def __init__(self, depth=0, max_depth=7):
        self.left = None
        self.right = None
        self.feat = None
        self.value = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None

    def train(self, X_train):
        features = X_train.columns[:-1]  # Exclude the target variable
        i_gain = []

        for feat in features:
            i_gain.append(info_gain(X_train, feat, X_train[feat].mean()))

        self.feat = features[np.argmax(i_gain)]
        self.value = X_train[self.feat].mean()

        data_left, data_right = divide_data(X_train, self.feat, self.value)

        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            self.target = X_train['quality'].mode()[0]
            return

        if self.depth >= self.max_depth:
            self.target = X_train['quality'].mode()[0]
            return

        self.left = Decision_Tree(depth=self.depth + 1, max_depth=self.max_depth)
        self.left.train(data_left)

        self.right = Decision_Tree(depth=self.depth + 1, max_depth=self.max_depth)
        self.right.train(data_right)

        self.target = X_train['quality'].mode()[0]

    def predict(self, test):
        if self.target is not None:
            return self.target

        if test[self.feat] >= self.value:
            return self.right.predict(test) if self.right else self.target
        else:
            return self.left.predict(test) if self.left else self.target

In [11]:
tree = Decision_Tree(max_depth = 3)
tree.train(X_train)

In [12]:
pred = X_test.apply(tree.predict, axis = 1)

In [13]:
from sklearn.metrics import accuracy_score

In [15]:
print('Accuracy:', accuracy_score(Y_test, pred) * 100)

Accuracy: 40.625
